<a href="https://colab.research.google.com/github/MiM0ulay/RiskMetrics/blob/main/GOLDXAURiskMetricFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install nasdaq-data-link


In [13]:
from datetime import date
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import nasdaqdatalink
import yfinance as yf

In [14]:

# Download data
df = yf.download(tickers='GC=F', start='2000-01-01', interval='1d')

# Debugging: Check the column names
print("Column names before processing:", df.columns)

# Reset the index and flatten column names (if needed)
df.reset_index(inplace=True)

# Handle multi-level columns (flatten them if they exist)
df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

# Debugging: Check flattened column names
print("Column names after flattening:", df.columns)

# Rename columns for consistency
if 'Open' in df.columns:
    df.rename(columns={'Date': 'date', 'Open': 'value'}, inplace=True)
elif 'value' not in df.columns:
    raise KeyError("The required columns ('date', 'value') are not found in the DataFrame.")

# Keep necessary columns
df = df[['date', 'value']].copy()

# Sort by date
df.sort_values(by='date', inplace=True)

# Calculate moving average
moving_average_days = 365
df['MA'] = df['value'].rolling(moving_average_days, min_periods=1).mean()

# Ensure no NaN values
df = df.dropna().reset_index(drop=True)

# Calculate Preavg
diminishing_factor = 0.395
df['Preavg'] = (np.log(df['value']) - np.log(df['MA'])) * (np.arange(len(df)) + 1) ** diminishing_factor

# Print results
print(df.head())


[*********************100%***********************]  1 of 1 completed

Column names before processing: MultiIndex([('Adj Close', 'GC=F'),
            (    'Close', 'GC=F'),
            (     'High', 'GC=F'),
            (      'Low', 'GC=F'),
            (     'Open', 'GC=F'),
            (   'Volume', 'GC=F')],
           names=['Price', 'Ticker'])
Column names after flattening: Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
        date       value          MA    Preavg
0 2000-08-30  273.899994  273.899994  0.000000
1 2000-08-31  274.799988  274.349991  0.002155
2 2000-09-01  277.000000  275.233327  0.009875
3 2000-09-05  275.799988  275.374992  0.002666
4 2000-09-06  274.200012  275.139996 -0.006463


In [15]:
print("Column names:", df.columns)


Column names: Index(['date', 'value', 'MA', 'Preavg'], dtype='object')


In [16]:
# Normalization to 0-1 range
df['avg'] = (df['Preavg'] - df['Preavg'].cummin()) / (df['Preavg'].cummax() - df['Preavg'].cummin())

In [17]:
df.head()

,date,value,MA,Preavg,avg
0,2000-08-30,273.899994,273.899994,0.000000,NaN
1,2000-08-31,274.799988,274.349991,0.002155,1.000000
2,2000-09-01,277.000000,275.233327,0.009875,1.000000
3,2000-09-05,275.799988,275.374992,0.002666,0.270029
4,2000-09-06,274.200012,275.139996,-0.006463,0.000000


In [18]:
price_per_risk = {
    round(risk, 1):round(np.exp(
        (risk * (df['Preavg'].cummax().iloc[-1] - (cummin := df['Preavg'].cummin().iloc[-1])) + cummin) / df.index[-1]**diminishing_factor + np.log(df['MA'].iloc[-1])
    ))
    for risk in np.arange(0.0, 1.0, 0.1)
}

In [19]:
# # Exclude the first 1000 days from the dataframe, because it's pure chaos
AnnotationText = f"Updated: {df.index[-1]} | Price: {round(df['value'].iloc[-1])} | Risk: {round(df['avg'].iloc[-1], 2)}"

In [20]:
# Plot BTC-USD and Risk on a logarithmic chart
fig = make_subplots(specs=[[{'secondary_y': True}]])

# Add BTC-USD and Risk data to the figure
fig.add_trace(go.Scatter(x=df['date'], y=df['value'], name='Price', line=dict(color='gold')))
fig.add_trace(go.Scatter(x=df['date'], y=df['avg'],   name='Risk',  line=dict(color='white')), secondary_y=True)

# Add green (`accumulation` or `buy`) rectangles to the figure
opacity = 0.2
for i in range(5, 0, -1):
    opacity += 0.05
    fig.add_hrect(y0=i*0.1, y1=((i-1)*0.1), line_width=0, fillcolor='green', opacity=opacity, secondary_y=True)

# Add red (`distribution` or `sell`) rectangles to the figure
opacity = 0.2
for i in range(6, 10):
    opacity += 0.1
    fig.add_hrect(y0=i*0.1, y1=((i+1)*0.1), line_width=0, fillcolor='red', opacity=opacity, secondary_y=True)

fig.update_xaxes(title='Date')
fig.update_yaxes(title='Price ($USD)', type='log', showgrid=False)
fig.update_yaxes(title='Risk', type='linear', secondary_y=True, showgrid=True, tick0=0.0, dtick=0.1, range=[0, 1])
fig.update_layout(template='plotly_dark', title={'text': AnnotationText, 'y': 0.9, 'x': 0.5})
fig.show()

# Plot BTC-USD colored according to Risk values on a logarithmic chart
fig = px.scatter(df, x='date', y='value', color='avg', color_continuous_scale='jet')
fig.update_yaxes(title='Price ($USD)', type='log', showgrid=False)
fig.update_layout(template='plotly_dark', title={'text': AnnotationText, 'y': 0.9, 'x': 0.5})
fig.show()

# Plot Predicting BTC price according to specific risk
fig = go.Figure(data=[go.Table(
    header=dict(values=['Risk', 'Price'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[list(price_per_risk.keys()), list(price_per_risk.values())],
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])
fig.update_layout(width=500, height=500, title={'text': 'Price according to specific risk', 'y': 0.9, 'x': 0.5})
fig.show()

In [21]:
df.tail(60)

,date,value,MA,Preavg,avg
6044,2024-10-03,2642.800049,2137.229587,6.616982,0.880319
6045,2024-10-04,2656.000000,2139.078080,6.745748,0.888476
6046,2024-10-07,2648.699951,2140.867121,6.634348,0.881419
6047,2024-10-08,2639.000000,2142.623833,6.494856,0.872582
6048,2024-10-09,2603.000000,2144.301641,6.042719,0.843938
6049,2024-10-10,2602.500000,2145.988217,6.012615,0.842031
6050,2024-10-11,2638.300049,2147.791505,6.412757,0.867380
6051,2024-10-14,2655.000000,2149.600546,6.583656,0.878207
6052,2024-10-15,2661.399902,2151.365751,6.633561,0.881369
6053,2024-10-16,2674.000000,2153.089039,6.756307,0.889145
